## Environment Setup

### Library Installation

In [1]:
!pip install pyspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u462-ga%7eus1-0ubuntu2%7e22.04.2_amd64.deb  404  Not Found [IP: 185.125.190.39 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jdk-headless_8u462-ga%7eus1-0ubuntu2%7e22.04.2_amd64.deb  404  Not Found [IP: 185.125.190.39 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


### Library Import

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, year
import time

## Data Loading and Preprocessing

In [3]:
!pip install -q kaggle

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!mkdir -p ~/.kaggle

In [6]:
!cp /content/drive/MyDrive/CollabData/kaggle_API/kaggle.json ~/.kaggle/kaggle.json

In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download har5hdeep5harma/chicago-crime-incidents-2001-to-present

Dataset URL: https://www.kaggle.com/datasets/har5hdeep5harma/chicago-crime-incidents-2001-to-present
License(s): other
 98% 508M/516M [00:03<00:00, 164MB/s]
100% 516M/516M [00:03<00:00, 179MB/s]


In [9]:
!unzip chicago-crime-incidents-2001-to-present.zip

Archive:  chicago-crime-incidents-2001-to-present.zip
  inflating: Chicago_Crimes_2001_to_Present.csv  


## Initialize Spark Session

In [11]:
print("Updating apt-get and installing openjdk-8-jdk-headless...")
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pyspark.sql import SparkSession
# These imports are also in IqeNjUwEFlc1, but included here for robustness if cell is run in isolation
from pyspark.sql.functions import col, to_timestamp, year
import time

spark = SparkSession.builder \
    .appName("StorageTechniques_ChicagoCrime") \
    .master("local[*]") \
    .getOrCreate()

print("Spark initialized")

Updating apt-get and installing openjdk-8-jdk-headless...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Spark initialized


## Load and Prepare Data

In [12]:
# Load the raw CSV
print("Loading CSV data...")
df_raw = spark.read.option("header", "true").option("inferSchema", "true").csv("Chicago_Crimes_2001_to_Present.csv")

# Clean up column names by removing spaces
new_columns = [c.replace(' ', '_') for c in df_raw.columns]
df = df_raw.toDF(*new_columns)

# Ensure Date format and Year column exist (used for partitioning)
df = df.withColumn("Year", col("Year").cast("integer"))

print(f"Total rows: {df.count()}")
df.printSchema()
df.show(5)

Loading CSV data...
Total rows: 8422096
root
 |-- ID: integer (nullable = true)
 |-- Case_Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary_Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location_Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community_Area: integer (nullable = true)
 |-- FBI_Code: string (nullable = true)
 |-- X_Coordinate: integer (nullable = true)
 |-- Y_Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated_On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)

+--------+-----------+--------------------+-------------

## Storage Comparison

### .csv

In [13]:
# Save as CSV
print("Writing CSV...")
start_time = time.time()
df.write.mode("overwrite").csv("chicago_data_csv")
print(f"CSV Write Time: {time.time() - start_time:.2f} seconds")

Writing CSV...
CSV Write Time: 113.33 seconds


### parquet

In [14]:
# Save as Parquet
print("Writing Parquet...")
start_time = time.time()
df.write.mode("overwrite").parquet("chicago_data_parquet")
print(f"Parquet Write Time: {time.time() - start_time:.2f} seconds")

Writing Parquet...
Parquet Write Time: 120.13 seconds


### Storage Comparison

In [15]:
# Compare Storage Size
print("\nSTORAGE SIZE COMPARISON")
!du -sh chicago_data_csv
!du -sh chicago_data_parquet


STORAGE SIZE COMPARISON
1.9G	chicago_data_csv
568M	chicago_data_parquet


## Partitioning Strategy

In [16]:
# Write Partitioned Parquet
print("Writing Partitioned Parquet (partitioned by Year)...")
start_time = time.time()
df.write.mode("overwrite").partitionBy("Year").parquet("chicago_data_partitioned")
print(f"Partitioned Write Time: {time.time() - start_time:.2f} seconds")

Writing Partitioned Parquet (partitioned by Year)...
Partitioned Write Time: 148.32 seconds


In [17]:
# Show folder structure
print("\nPARTITION STRUCTURE")
!ls -F chicago_data_partitioned | head -n 15


PARTITION STRUCTURE
_SUCCESS
Year=2001/
Year=2002/
Year=2003/
Year=2004/
Year=2005/
Year=2006/
Year=2007/
Year=2008/
Year=2009/
Year=2010/
Year=2011/
Year=2012/
Year=2013/
Year=2014/


## Query Performance Benchmarking

In [18]:
target_year = 2024

# Query 1: Non-Partitioned
# Spark must scan the entire dataset to find rows where Year = 2024
print(f"Querying Non-Partitioned Data for Year {target_year}...")
start = time.time()
count_raw = spark.read.parquet("chicago_data_parquet").filter(f"Year = {target_year}").count()
print(f"Time: {time.time() - start:.4f} seconds (Count: {count_raw})")

Querying Non-Partitioned Data for Year 2024...
Time: 3.4280 seconds (Count: 258734)


In [19]:
# Query 2: Partitioned
# Spark goes directly to the folder /Year=2024/ and reads only that data
print(f"Querying Partitioned Data for Year {target_year}...")
start = time.time()
count_part = spark.read.parquet("chicago_data_partitioned").filter(f"Year = {target_year}").count()
print(f"Time: {time.time() - start:.4f} seconds (Count: {count_part})")

Querying Partitioned Data for Year 2024...
Time: 0.7997 seconds (Count: 258734)


## Compression and Download

In [20]:
print("Compressing CSV output...")
!tar -czf chicago_data_csv.tar.gz chicago_data_csv

print("Compressing Parquet output...")
!tar -czf chicago_data_parquet.tar.gz chicago_data_parquet

print("Files ready for download:")
!ls -lh *.tar.gz

Compressing CSV output...
Compressing Parquet output...
Files ready for download:
-rw-r--r-- 1 root root 515M Nov 26 03:16 chicago_data_csv.tar.gz
-rw-r--r-- 1 root root 457M Nov 26 03:18 chicago_data_parquet.tar.gz
